In [40]:
import numpy as np
from random import randint
import math

### UTILS

In [41]:
def get_problem(NUM_KNAPSACKS, NUM_ITEMS, NUM_DIMENSIONS, VALUES, WEIGHTS, CONSTRAINTS):
    return {
        "NUM_KNAPSACKS": NUM_KNAPSACKS,
        "NUM_ITEMS": NUM_ITEMS,
        "NUM_DIMENSIONS": NUM_DIMENSIONS,
        "VALUES": VALUES,
        "WEIGHTS": WEIGHTS,
        "CONSTRAINTS": CONSTRAINTS,
    }

In [42]:
def generate_random_solution(num_knapsacks, num_items):
    """
    generate a pure random solution. In this case is possible to start
    from a starting point where you have items that are carried by more than 1 knapsack.
    """
    
    solution = np.array(
        [np.random.random(num_items) < 0.5 for _ in range(num_knapsacks)], dtype=np.int8
    )

    return solution

In [43]:
def generate_clean_random_solution(num_knapsacks, num_items):
    """
    generate a random solution that satisfy the first constraint
    (that is each item is either not carried or carried from exactly 1 item).
    """

    # generate a 2D matrix (num_knapsacks, num_items) full of 0s.
    solution = np.zeros((num_knapsacks, num_items), dtype=int)

    # iterate over the columns
    for item_idx in range(num_items):

        # generate random number:
        # if 0 the item is not carried
        # if 1 the item is carried by 1 knapsack
        carried = randint(0, 1)
    
        if carried == 0:
            continue
        
        # generate a random index that identifies the knapsack that carries the item
        id_knapsack = randint(0, num_knapsacks - 1)

        # update the solution
        solution[id_knapsack, item_idx] = 1
        
    return solution

In [44]:
def fitness(solution, problem):
    """
    Calculates the fitness of a solution. It is defined by the overall valued
    carried by all knapsacks.
    """

    # check the basic constraint: an item cannot be carried by more then 1 knapsack
    # if the sum of the items in the same column >= 1 it means that
    # at leats 2 knapsacks are carrying the item, so a fitness of 0 is returned
    if np.any(np.sum(solution, axis=0) > 1):
        return 0

    values = problem["VALUES"]
    weights = problem["WEIGHTS"]
    constraints = problem["CONSTRAINTS"]

    # sum all the values for each row based on the items that each knapsack carries
    total_value = np.sum(solution * values)
    # get the total weights for each knapsack
    knapsack_loads = solution @ weights

    # Check if any constraint is violated
    if np.any(knapsack_loads > constraints):
        return 0 
    else:
        return total_value

In [45]:
def tweak(solution, problem):
    """
    Creates a "neighbor" solution by making one small, random change.
    It randomly picks one item and changes its knapsack assignment.
    """
    # get problem parameters
    num_items = problem["NUM_ITEMS"]
    num_knapsacks = problem["NUM_KNAPSACKS"]
    
    # create a copy to avoid modifying the original solution
    neighbor_solution = solution.copy()
    
    # pick a random item to move
    item_to_move = np.random.randint(num_items)
    # pick a random new knapsack for it (-1 means removing it)
    new_knapsack = np.random.randint(-1, num_knapsacks)
    
    # first, remove the item from its old position (if any)
    neighbor_solution[:, item_to_move] = 0
    # if the item is being assigned to a new knapsack (not just removed)
    if new_knapsack != -1:
        # place the item in the new knapsack
        neighbor_solution[new_knapsack, item_to_move] = 1
        
    return neighbor_solution

### ALGORITHMS

A new version of hill climbing has been implemented
This version of hill climbing solves 1 constraint at the time. It is organized in 2 phases.
- PHASE 1: It first make sure that the solution satisfies a specific constraint, then we keep iterating the process checking the solution for all the other constraint.
- PHASE 2: The hill climbing algorithm is applied an iteratively looks for a slightly better solution by making a tweak of only 1 element.

In [ ]:
def guided_hill_climbing(problem, max_steps=10000):
    """
    This version of hill climbing focuses on 1 constraint at the time.
    It first repairs the solution to make it valid, then optimizes it for value.
    """
    num_knapsacks = problem["NUM_KNAPSACKS"]
    num_items = problem["NUM_ITEMS"]
    weights = problem["WEIGHTS"]
    values = problem["VALUES"]
    constraints = problem["CONSTRAINTS"]
    
    # start with a random solution
    current_solution = generate_clean_random_solution(num_knapsacks, num_items)
    
    # PHASE 1
    # taking an invalid solution and repair it
    print("Starting Repair Phase...")
    
    # loop until the solution becomes valid
    while True:
        current_loads = current_solution @ weights
        overload = current_loads - constraints
        overweight_knapsacks = np.where(np.any(overload > 0, axis=1))[0]
        
        if len(overweight_knapsacks) == 0:
            print("Solution is now valid. Moving to Optimization Phase.")
            break 
            
        knapsack_to_fix = overweight_knapsacks[0]
        items_in_knapsack = np.where(current_solution[knapsack_to_fix] == 1)[0]
        
        if len(items_in_knapsack) == 0:
            continue

        item_to_remove = np.random.choice(items_in_knapsack)
        current_solution[knapsack_to_fix, item_to_remove] = 0
        
    # PHASE 2 - Suggested version
    # now the solution is valid, so we optimize it
    current_fitness = fitness(current_solution, problem)
    print(f"Initial valid fitness: {current_fitness}")
    
    # start the optimization loop for a fixed number of steps
    for i in range(max_steps):
        # *** REFACTORED PART ***
        # create a neighbor solution by calling the new tweak function
        neighbor_solution = tweak(current_solution, problem)

        #Calculate the neighbor's fitness using the fitness function.
        # This function returns 0 if the solution is invalid (violates constraints),
        # automatically handling both validity and optimization.
        neighbor_fitness = fitness(neighbor_solution, problem)
        
        # check if the neighbor's fitness is better than our current best
        if neighbor_fitness > current_fitness:
            current_solution = neighbor_solution
            current_fitness = neighbor_fitness
            print(f"Step {i}: Found better solution with fitness {current_fitness}")

    final_fitness = fitness(current_solution, problem)
    return current_solution, final_fitness

### TEST PROBLEMS

In [47]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

# getting problem p1
p1 = get_problem(NUM_KNAPSACKS, NUM_ITEMS, NUM_DIMENSIONS, VALUES, WEIGHTS, CONSTRAINTS)

In [48]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

p2 = get_problem(NUM_KNAPSACKS, NUM_ITEMS, NUM_DIMENSIONS, VALUES, WEIGHTS, CONSTRAINTS)

In [49]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

p3 = get_problem(NUM_KNAPSACKS, NUM_ITEMS, NUM_DIMENSIONS, VALUES, WEIGHTS, CONSTRAINTS)

### FINAL TEST - GUIDED HILL CLIMBING algorithm

In [50]:
print("PROBLEM 1")
best_solution, best_fitness = guided_hill_climbing(p1)

print(f"Best fitness: {best_fitness}")
print(f"Best solution:\n {best_solution}")

PROBLEM 1
Starting Repair Phase...
Solution is now valid. Moving to Optimization Phase.


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
print("PROBLEM 2")
best_solution, best_fitness = guided_hill_climbing(p2)

print(f"Best fitness: {best_fitness}")
print(f"Best solution:\n {best_solution}")

PROBLEM 2
Starting Repair Phase...
Solution is now valid. Moving to Optimization Phase.
Initial valid fitness: 19720
Step 2: Found better solution with fitness 20090
Step 6: Found better solution with fitness 20825
Step 8: Found better solution with fitness 21569
Step 15: Found better solution with fitness 21939
Step 16: Found better solution with fitness 22485
Step 19: Found better solution with fitness 22582
Step 24: Found better solution with fitness 23082
Step 25: Found better solution with fitness 23849
Step 26: Found better solution with fitness 24531
Step 31: Found better solution with fitness 25335
Step 33: Found better solution with fitness 25411
Step 39: Found better solution with fitness 25908
Step 41: Found better solution with fitness 26036
Step 42: Found better solution with fitness 26218
Step 43: Found better solution with fitness 26307
Step 46: Found better solution with fitness 26694
Step 53: Found better solution with fitness 27006
Step 59: Found better solution with 

In [ ]:
print("PROBLEM 3")
best_solution, best_fitness = guided_hill_climbing(p3, max_steps=1000)

print(f"Best fitness: {best_fitness}")
# print(f"Best solution:\n {best_solution}")

PROBLEM 3
Starting Repair Phase...
Solution is now valid. Moving to Optimization Phase.
Initial valid fitness: 963673
Step 1: Found better solution with fitness 964528
Step 2: Found better solution with fitness 965276
Step 8: Found better solution with fitness 965583
Step 10: Found better solution with fitness 965591
Step 12: Found better solution with fitness 966160
Step 14: Found better solution with fitness 966442
Step 19: Found better solution with fitness 966597
Step 21: Found better solution with fitness 967355
Step 29: Found better solution with fitness 967365
Step 30: Found better solution with fitness 968036
Step 31: Found better solution with fitness 968506
Step 37: Found better solution with fitness 968543
Step 38: Found better solution with fitness 968696
Step 41: Found better solution with fitness 969222
Step 44: Found better solution with fitness 969774
Step 47: Found better solution with fitness 970390
Step 48: Found better solution with fitness 971211
Step 52: Found bet